In [44]:
#importing important libraries and modules and defining dataframe
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
df = pd.read_csv('car data.csv')
print(df)


    Car_Name  Year  Selling_Price  Present_Price  Driven_kms Fuel_Type  \
0       ritz  2014           3.35           5.59       27000    Petrol   
1        sx4  2013           4.75           9.54       43000    Diesel   
2       ciaz  2017           7.25           9.85        6900    Petrol   
3    wagon r  2011           2.85           4.15        5200    Petrol   
4      swift  2014           4.60           6.87       42450    Diesel   
..       ...   ...            ...            ...         ...       ...   
296     city  2016           9.50          11.60       33988    Diesel   
297     brio  2015           4.00           5.90       60000    Petrol   
298     city  2009           3.35          11.00       87934    Petrol   
299     city  2017          11.50          12.50        9000    Diesel   
300     brio  2016           5.30           5.90        5464    Petrol   

    Selling_type Transmission  Owner  
0         Dealer       Manual      0  
1         Dealer       Manual    

In [45]:
#encoding all the non numeric columns in the df
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to non-numeric columns
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])
print(df)

     Car_Name  Year  Selling_Price  Present_Price  Driven_kms  Fuel_Type  \
0          90  2014           3.35           5.59       27000          2   
1          93  2013           4.75           9.54       43000          1   
2          68  2017           7.25           9.85        6900          2   
3          96  2011           2.85           4.15        5200          2   
4          92  2014           4.60           6.87       42450          1   
..        ...   ...            ...            ...         ...        ...   
296        69  2016           9.50          11.60       33988          1   
297        66  2015           4.00           5.90       60000          2   
298        69  2009           3.35          11.00       87934          2   
299        69  2017          11.50          12.50        9000          1   
300        66  2016           5.30           5.90        5464          2   

     Selling_type  Transmission  Owner  
0               0             1      0  
1    

In [46]:
#Define training Set
X =  df.loc[:, ~df.columns.isin(['Selling_Price'])].values  # All columns except the selling price
y = df.iloc[:, 2].values   # Only the selling price

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (301, 8)
Shape of y: (301,)


In [47]:
#we see that the features of X are in different ranges so we scale the features of X so they can be in the same range
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
scaler = StandardScaler()
x = scaler.fit_transform(X)
#splitting training set for model evaluation after fitting is done to check for bias(underfitting) or variance(overfitting)
from sklearn.model_selection import train_test_split

# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")


the shape of the training set (input) is: (180, 8)
the shape of the training set (target) is: (180,)

the shape of the cross validation set (input) is: (60, 8)
the shape of the cross validation set (target) is: (60,)

the shape of the test set (input) is: (61, 8)
the shape of the test set (target) is: (61,)


In [48]:
#increasing the training set size because the more training examples we have, the better our algorithm learns the training set
#so we duplicate the input and target array by a 100 times
Xt = np.tile(x_train,(100,1))
Yt= np.tile(y_train,(100))   
print(Xt.shape, Yt.shape)

(18000, 8) (18000,)


In [49]:
#define tensorflow model with one hidden layer with three activations and activation function of relu(0 for the negative values
#of z and linear for the positive values of z )
#and an output layer with one activation representing the selling price and an activation function of linear(values of
#z remain the sdme)
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(8,)),
        Dense(3, activation='relu', name = 'layer1'),
        Dense(1, activation='linear', name = 'layer2')
     ]
)
model.summary()
#shows you a breakdown of the parameters

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 3)                 27        
_________________________________________________________________
layer2 (Dense)               (None, 1)                 4         
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


In [50]:
#define the loss function which is the mean squared error for linear regression
#we define the learning rate and use the adam optimizer which dynamically adjusts the learning rate when training


model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)
#fit the model to this data
model.fit(
    Xt,Yt,            
    epochs=10,
)

Train on 18000 samples
Epoch 1/10
18000/18000 [==============================] - 2s 105us/sample - loss: 6.6396
Epoch 2/10
18000/18000 [==============================] - 1s 58us/sample - loss: 0.9880
Epoch 3/10
18000/18000 [==============================] - 1s 59us/sample - loss: 0.6590
Epoch 4/10
18000/18000 [==============================] - 1s 58us/sample - loss: 0.6140
Epoch 5/10
18000/18000 [==============================] - 1s 65us/sample - loss: 0.5983
Epoch 6/10
18000/18000 [==============================] - 1s 81us/sample - loss: 0.5966
Epoch 7/10
18000/18000 [==============================] - 2s 85us/sample - loss: 0.5992
Epoch 8/10
18000/18000 [==============================] - 1s 81us/sample - loss: 0.5987
Epoch 9/10
18000/18000 [==============================] - 1s 77us/sample - loss: 0.5966
Epoch 10/10
18000/18000 [==============================] - 1s 64us/sample - loss: 0.5928


In [51]:
W1_norm, b1_norm = model.get_layer("layer1").get_weights() #the normalized weights and biases
W2_norm, b2_norm = model.get_layer("layer2").get_weights()
print("W1:\n", W1_norm, "\nb1:", b1_norm)
print("W2:\n", W2_norm, "\nb2:", b2_norm)

W1:
 [[ 0.3292535   0.96947753  1.2299649 ]
 [-0.499663    2.0780513   3.0503995 ]
 [ 1.0140072  -2.2137315   1.5880443 ]
 [ 0.28228396 -0.47331786 -0.7956308 ]
 [ 0.9388719  -0.5683351  -0.84688073]
 [-1.3616987  -0.7342973  -0.5245941 ]
 [ 1.2373908  -1.0367063  -1.3127818 ]
 [-0.7221538   0.27949357  0.49812976]] 
b1: [-0.11195582 -0.5359699   1.5092524 ]
W2:
 [[ 0.7669378]
 [-1.8653992]
 [ 1.7208183]] 
b2: [0.642372]


In [52]:
#making predictions with model parameters for the train,cv and test set to check for high bias(overfitting) and high variance
#(underfitting)
yhat_train = model.predict(x_train)
yhat_cv = model.predict(x_cv)
yhat_test = model.predict(x_test)

#calculating how well the model works on the train set and the cross validation set by finding the MSE(how much the prediction
#differs from the target) for the train and validation set
from sklearn.metrics import mean_squared_error
err_train = mean_squared_error(y_train, yhat_train) / 2
err_cv = mean_squared_error(y_cv, yhat_cv) / 2
err_test = mean_squared_error(y_test, yhat_test) / 2
print("train error",err_train)
print("cv error",err_cv)
print("test error", err_test)


train error 0.2961286344653438
cv error 0.30679257735089477
test error 0.7154994382967785


In [ ]:
#even though the corss-valiadtion error is slightly higher than the train error we see however model does very weell
#on all the different sets and therefore is a good fir for car prediction prices
#The input features of a new car to be predicted mustr first be normalized 